<a href="https://colab.research.google.com/github/aayeshaqureshi/nlp/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FAKE NEWS CLASSIFIER USING LSTM**

# New Section

1.   dataset
2. divide into dependent and independent features
3. cleaing the data:
4. stemming
5. stop words
6. fix a sentence length to fix the input
7. OHE
8. LSTM neural network






## Given a dataset from kaggle we need to predict if the news is fake or not. We are given its id, author, text and so on

In [ ]:
#download the dataset
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import re
data= pd.read_csv('/content/drive/MyDrive/train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# check for null values

In [ ]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Since this is a text data we can drop the nan values 
We have a hige amount of records so we can delete arund 2000 values


In [ ]:
data.shape

(20800, 5)

## Dropping the NaN values 

In [ ]:
data= data.dropna()

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1




*   title is the only feature we need to apply LSTM
*   drop all the other features
in order to do that we need to first remove the dependent feature which is "label"











In [ ]:
## get the independent features
x= data.drop('label', axis=1)

In [ ]:
## get the dependent features
## df of label will give you the dependent feature which is the ouput

In [ ]:
y = data['label']

In [ ]:
x.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

# IMPORT TENSOR FLOW

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__

'2.9.2'

# import libraries 

In [ ]:
from tensorflow.keras.layers import Embedding  #helps implement the word2vec
from tensorflow.keras.preprocessing.sequence import pad_sequences #used for padding- pre and post padding
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense







# VOCABULARY SIZE
vocabulary- number of unique words

In [ ]:
voc_size= 5000

# one hot encoding- at which index that particular word exists in the 5000 words

In [ ]:
# make a copy of x
messages= x.copy()

In [ ]:
#consider only title because this is the feature we use for training our model
messages['title'][4]


'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [ ]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
# reset the index
#messages.reset_index(inplace= True)

# STOP WORDS
# STEMMING


DOWNLOAD ALL THE STOPWORDS

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# PRE PROCESSING THE DATASET
porter stemmer is specifically used for stemming purpose

In [ ]:
from nltk.stem.porter import PorterStemmer 
ps= PorterStemmer()
corpus=[]
#loop through each and every data point inside the messages
#then pickup the title coloumn and apply stemming in each word
#then apply stop words

for i in range(0, len(messages)):
  #apart from a-z and A-Z remove all special chars
  review= re.sub('[^a-zA-Z]', ' ',messages['title'][i])
  #lower case
  review= review.lower()
  # to get word by word we use split
  review= review.split()
  review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
  #join all and combine
  review= ' '.join(review)
  corpus.append(review)





 
 




In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

# ONE HOT ENCODING

In [ ]:
onehot_rep= [one_hot(words, voc_size) for words in corpus]

In [ ]:
onehot_rep

[[4631, 3578, 74, 3643, 1927, 2660, 269, 4874, 3850, 4889],
 [496, 2471, 2947, 2217, 540, 1821, 978],
 [1854, 2154, 3744, 3416],
 [1716, 1839, 3902, 4979, 3836, 1792],
 [4240, 540, 2458, 515, 1523, 3732, 540, 280, 2006, 3140],
 [1988,
  4422,
  4922,
  2065,
  221,
  4686,
  3071,
  968,
  3999,
  3068,
  2857,
  1672,
  4617,
  1329,
  978],
 [580, 4092, 1333, 4623, 751, 3971, 2755, 2244, 4839, 4072, 908],
 [3315, 185, 2230, 4894, 2539, 1066, 4686, 1834, 4839, 4072, 908],
 [2060, 4196, 3242, 3302, 1886, 2094, 185, 3216, 4686, 1427],
 [2625, 2036, 1343, 289, 2773, 4266, 377, 338],
 [977, 2800, 2376, 2733, 538, 2184, 1106, 4072, 958, 948, 2970],
 [4979, 725, 1927, 2094, 4686, 2539],
 [3513, 436, 1957, 73, 4917, 2972, 1335, 243, 1962],
 [2302, 4034, 1992, 2366, 66, 2750, 2719, 4839, 4072, 908],
 [2532, 4766, 2408, 985, 897, 4839, 4072, 908],
 [3166, 2386, 4203, 1273, 2854, 3998, 1051, 448, 3109, 1756],
 [99, 506, 2471],
 [2236, 1660, 4939, 2937, 4686, 4564, 2011, 978],
 [2631, 1367, 2947

In [ ]:
corpus[1] 


'flynn hillari clinton big woman campu breitbart'

In [ ]:
# the corpus 1 and the ohe of corpus 1
 
onehot_rep[1] 


[496, 2471, 2947, 2217, 540, 1821, 978]

# WORD EMBEDDING REPRESENTATION OR WORD2VEC REPRESENTAION

we need to make sure the length of all the sentences should be same- we use padding. librarry- pad sequences 

In [ ]:
# every sentence should be of len 20
sent_length= 20
embedded_docs= pad_sequences(onehot_rep, padding= 'pre', maxlen= sent_length)
print(embedded_docs)

[[   0    0    0 ... 4874 3850 4889]
 [   0    0    0 ...  540 1821  978]
 [   0    0    0 ... 2154 3744 3416]
 ...
 [   0    0    0 ... 4839 4072  908]
 [   0    0    0 ... 3259 2168 3312]
 [   0    0    0 ... 1236 2755 4145]]


In [ ]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  496, 2471, 2947, 2217,  540, 1821,  978], dtype=int32)

# CREATING A MODEL- Apply the LSTM and embedding layer 


In [ ]:
#each word is converted into 40 vector features
#voc size is 500
#sent length=20
#embedding layer
embedding_vectors_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vectors_features, input_length= sent_length))
#lstm neural network with 100 neaurons
model.add(LSTM(100))
#output is binary so activation function is sigmoid
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
len(embedded_docs), y.shape

(18285, (18285,))

## convert all embedded docs into arrays

In [ ]:
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [ ]:
x_final.shape

(18285, 20)

In [ ]:
y_final.shape

(18285,)

# TRAIN TEST AND SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train, y_test= train_test_split(x_final, y_final, test_size=30, random_state=4) 

# FIT MODEL BY RUNNING 10 EPOCHS 

In [ ]:
model.fit(x_train, y_train, validation_data= (x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
286/286 [==============================] - 8s 22ms/step - loss: 0.2863 - accuracy: 0.8664 - val_loss: 0.2466 - val_accuracy: 0.9000
Epoch 2/10
286/286 [==============================] - 6s 20ms/step - loss: 0.1400 - accuracy: 0.9457 - val_loss: 0.2557 - val_accuracy: 0.9333
Epoch 3/10
286/286 [==============================] - 6s 20ms/step - loss: 0.1104 - accuracy: 0.9588 - val_loss: 0.2820 - val_accuracy: 0.8333
Epoch 4/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0917 - accuracy: 0.9656 - val_loss: 0.2727 - val_accuracy: 0.8333
Epoch 5/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0746 - accuracy: 0.9730 - val_loss: 0.3743 - val_accuracy: 0.8667
Epoch 6/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0575 - accuracy: 0.9794 - val_loss: 0.4329 - val_accuracy: 0.9000
Epoch 7/10
286/286 [==============================] - 6s 20ms/step - loss: 0.0388 - accuracy: 0.9867 - val_loss: 0.4085 - val_accuracy: 0.9333

# performance metrics and accuracy

In [ ]:
# predict wrt to x_test
y_pred= model.predict(x_test)

1/1 [==============================] - 0s 17ms/step


In [ ]:
y_pred=np.where(y_pred>0.5, 1,0)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

array([[12,  3],
       [ 2, 13]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8333333333333334

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83        15
           1       0.81      0.87      0.84        15

    accuracy                           0.83        30
   macro avg       0.83      0.83      0.83        30
weighted avg       0.83      0.83      0.83        30

